Data
Loading[<u>¶</u>](https://www.kaggle.com/code/sillankumar/clean-vs-dirty-plate-img-data/notebook#Data-Loading)

In \[1\]:

*\# This Python 3 environment comes with many helpful analytics
libraries installed*

*\# It is defined by the kaggle/python Docker image:
https://github.com/kaggle/docker-python*

*\# For example, here's several helpful packages to load*

import numpy as np *\# linear algebra*

import pandas as pd *\# data processing, CSV file I/O (e.g.
pd.read_csv)*

*\# Input data files are available in the read-only "../input/"
directory*

*\# For example, running this (by clicking run or pressing Shift+Enter)
will list all files under the input directory*

import os

for dirname, \_, filenames **in** os.walk('/kaggle/input'):

for filename **in** filenames:

print(os.path.join(dirname, filename))

*\# You can write up to 20GB to the current directory (/kaggle/working/)
that gets preserved as output when you create a version using "Save &
Run All"*

*\# You can also write temporary files to /kaggle/temp/, but they won't
be saved outside of the current session*

/kaggle/input/platesv2/sample_submission.csv

/kaggle/input/platesv2/plates.zip

In \[2\]:

from zipfile import ZipFile

In \[3\]:

file='/kaggle/input/platesv2/plates.zip'

In \[4\]:

with ZipFile(file,'r') as zip:

*#zip.printdir()*

zip.extractall('/kaggle/working/')

In \[5\]:

print(os.listdir('/kaggle/working/'))

data_root = '/kaggle/working/plates/'

print(data_root)

print(os.listdir(data_root))

\['\_\_MACOSX', '\_\_notebook\_\_.ipynb', 'plates'\]

/kaggle/working/plates/

\['.DS_Store', 'test', 'train'\]

In \[6\]:

path_train = "/kaggle/working/plates/train/"

path_test = "/kaggle/working/plates/"

Image Augmentation

In \[7\]:

import tensorflow as tf

from tensorflow.keras.preprocessing.image import
ImageDataGenerator,img_to_array,load_img

In \[8\]:

validation_split = 0.2

train_datagen=ImageDataGenerator(validation_split=validation_split,rescale=1./255,rotation_range=45,shear_range=0.2,width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')

validation_datagen=ImageDataGenerator(validation_split=validation_split
,rescale=1./255)

test_datagen=ImageDataGenerator(rescale=1./255)

In \[9\]:

train_generator=train_datagen.flow_from_directory(directory=path_train,classes=\['cleaned','dirty'\],batch_size=64,class_mode='binary',shuffle=True,target_size=(224,224),subset='training')

validation_generator=validation_datagen.flow_from_directory(directory=path_train,classes=\['cleaned','dirty'\],batch_size=64,class_mode='binary',shuffle=True,target_size=(224,224),subset='validation')

test_generator=test_datagen.flow_from_directory(directory=path_test,classes=\['test'\],batch_size
= 1,shuffle = False,class_mode=None,target_size=(224,224))

Found 32 images belonging to 2 classes.

Found 8 images belonging to 2 classes.

Found 744 images belonging to 1 classes.

Model VGG16

In \[10\]:

from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import
Dense,InputLayer,BatchNormalization,Dropout,Flatten

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In \[11\]:

basemodel=VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

Downloading data from
https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

**58889256/58889256** ━━━━━━━━━━━━━━━━━━━━ **0s** 0us/step

In \[12\]:

for layer **in** basemodel.layers:

layer.trainable=False

In \[13\]:

model=Sequential()

model.add(basemodel)

model.add(Flatten())

model.add(Dense(512,activation='relu',
kernel_regularizer=tf.keras.regularizers.l2(0.01)))

model.add(Dropout(0.6))

model.add(BatchNormalization())

model.add(Dense(1,activation='sigmoid'))

In \[14\]:

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=\['accuracy'\])

In \[15\]:

earlystopping=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
patience=5, min_lr=1e-6)

In \[16\]:

history=model.fit(train_generator,epochs=100,validation_data=validation_generator,callbacks=\[earlystopping,reduce_lr\],verbose=1)

Epoch 1/100

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121:
UserWarning: Your \`PyDataset\` class should call
\`super().\_\_init\_\_(\*\*kwargs)\` in its constructor. \`\*\*kwargs\`
can include \`workers\`, \`use_multiprocessing\`, \`max_queue_size\`. Do
not pass these arguments to \`fit()\`, as they will be ignored.

self.\_warn_if_super_not_called()

**1/1** ━━━━━━━━━━━━━━━━━━━━ **16s** 16s/step - accuracy: 0.4688 - loss:
10.8902 - val_accuracy: 0.5000 - val_loss: 9.7651 - learning_rate:
0.0010

Epoch 2/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.6875 - loss:
9.4149 - val_accuracy: 0.5000 - val_loss: 9.7890 - learning_rate: 0.0010

Epoch 3/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **20s** 20s/step - accuracy: 0.5000 - loss:
8.5989 - val_accuracy: 0.5000 - val_loss: 9.5840 - learning_rate: 0.0010

Epoch 4/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.5938 - loss:
7.5469 - val_accuracy: 0.5000 - val_loss: 9.2695 - learning_rate: 0.0010

Epoch 5/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.7500 - loss:
6.6956 - val_accuracy: 0.5000 - val_loss: 8.6106 - learning_rate: 0.0010

Epoch 6/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.7812 - loss:
6.0688 - val_accuracy: 0.5000 - val_loss: 7.9346 - learning_rate: 0.0010

Epoch 7/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.7188 - loss:
5.4994 - val_accuracy: 0.5000 - val_loss: 7.2504 - learning_rate: 0.0010

Epoch 8/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8750 - loss:
4.9430 - val_accuracy: 0.5000 - val_loss: 6.7382 - learning_rate: 0.0010

Epoch 9/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8438 - loss:
4.4180 - val_accuracy: 0.5000 - val_loss: 6.2085 - learning_rate: 0.0010

Epoch 10/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.6875 - loss:
4.3306 - val_accuracy: 0.3750 - val_loss: 5.6031 - learning_rate: 0.0010

Epoch 11/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.6562 - loss:
4.0848 - val_accuracy: 0.3750 - val_loss: 5.1548 - learning_rate: 0.0010

Epoch 12/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.6875 - loss:
3.7628 - val_accuracy: 0.3750 - val_loss: 4.8280 - learning_rate: 0.0010

Epoch 13/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.6562 - loss:
3.6338 - val_accuracy: 0.3750 - val_loss: 4.5644 - learning_rate: 0.0010

Epoch 14/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8125 - loss:
3.3728 - val_accuracy: 0.3750 - val_loss: 4.3259 - learning_rate: 0.0010

Epoch 15/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8438 - loss:
3.2297 - val_accuracy: 0.3750 - val_loss: 4.1578 - learning_rate: 0.0010

Epoch 16/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **20s** 20s/step - accuracy: 0.8438 - loss:
3.1219 - val_accuracy: 0.3750 - val_loss: 4.0202 - learning_rate: 0.0010

Epoch 17/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8750 - loss:
3.0321 - val_accuracy: 0.3750 - val_loss: 3.9291 - learning_rate: 0.0010

Epoch 18/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8125 - loss:
2.9848 - val_accuracy: 0.3750 - val_loss: 3.7812 - learning_rate: 0.0010

Epoch 19/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8125 - loss:
2.9240 - val_accuracy: 0.3750 - val_loss: 3.6545 - learning_rate: 0.0010

Epoch 20/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.6562 - loss:
3.0307 - val_accuracy: 0.5000 - val_loss: 3.4949 - learning_rate: 0.0010

Epoch 21/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8438 - loss:
2.8080 - val_accuracy: 0.6250 - val_loss: 3.3698 - learning_rate: 0.0010

Epoch 22/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.7500 - loss:
2.8243 - val_accuracy: 0.5000 - val_loss: 3.3060 - learning_rate: 0.0010

Epoch 23/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **13s** 13s/step - accuracy: 0.8438 - loss:
2.6465 - val_accuracy: 0.5000 - val_loss: 3.2344 - learning_rate: 0.0010

Epoch 24/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8750 - loss:
2.5514 - val_accuracy: 0.5000 - val_loss: 3.1685 - learning_rate: 0.0010

Epoch 25/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8125 - loss:
2.5000 - val_accuracy: 0.5000 - val_loss: 3.1023 - learning_rate: 0.0010

Epoch 26/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8125 - loss:
2.5978 - val_accuracy: 0.3750 - val_loss: 3.0464 - learning_rate: 0.0010

Epoch 27/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.7812 - loss:
2.4375 - val_accuracy: 0.3750 - val_loss: 2.9848 - learning_rate: 0.0010

Epoch 28/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8750 - loss:
2.3054 - val_accuracy: 0.5000 - val_loss: 2.9184 - learning_rate: 0.0010

Epoch 29/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8125 - loss:
2.3376 - val_accuracy: 0.5000 - val_loss: 2.8329 - learning_rate: 0.0010

Epoch 30/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8125 - loss:
2.2301 - val_accuracy: 0.3750 - val_loss: 2.7473 - learning_rate: 0.0010

Epoch 31/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9062 - loss:
2.0937 - val_accuracy: 0.3750 - val_loss: 2.6810 - learning_rate: 0.0010

Epoch 32/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8750 - loss:
1.9812 - val_accuracy: 0.3750 - val_loss: 2.6214 - learning_rate: 0.0010

Epoch 33/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.7812 - loss:
2.1366 - val_accuracy: 0.3750 - val_loss: 2.5735 - learning_rate: 0.0010

Epoch 34/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8438 - loss:
1.9317 - val_accuracy: 0.3750 - val_loss: 2.5224 - learning_rate: 0.0010

Epoch 35/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8750 - loss:
1.8880 - val_accuracy: 0.3750 - val_loss: 2.4795 - learning_rate: 0.0010

Epoch 36/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9062 - loss:
1.8237 - val_accuracy: 0.3750 - val_loss: 2.4342 - learning_rate: 0.0010

Epoch 37/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **20s** 20s/step - accuracy: 0.8750 - loss:
1.8598 - val_accuracy: 0.3750 - val_loss: 2.3755 - learning_rate: 0.0010

Epoch 38/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9062 - loss:
1.7464 - val_accuracy: 0.3750 - val_loss: 2.3316 - learning_rate: 0.0010

Epoch 39/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **20s** 20s/step - accuracy: 0.8125 - loss:
1.7679 - val_accuracy: 0.3750 - val_loss: 2.2763 - learning_rate: 0.0010

Epoch 40/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8438 - loss:
1.7252 - val_accuracy: 0.3750 - val_loss: 2.2113 - learning_rate: 0.0010

Epoch 41/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.7188 - loss:
1.7284 - val_accuracy: 0.3750 - val_loss: 2.1568 - learning_rate: 0.0010

Epoch 42/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8438 - loss:
1.5882 - val_accuracy: 0.3750 - val_loss: 2.1132 - learning_rate: 0.0010

Epoch 43/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9688 - loss:
1.4736 - val_accuracy: 0.3750 - val_loss: 2.0740 - learning_rate: 0.0010

Epoch 44/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9062 - loss:
1.5323 - val_accuracy: 0.3750 - val_loss: 2.0373 - learning_rate: 0.0010

Epoch 45/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8438 - loss:
1.5492 - val_accuracy: 0.3750 - val_loss: 1.9996 - learning_rate: 0.0010

Epoch 46/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 1.0000 - loss:
1.3777 - val_accuracy: 0.3750 - val_loss: 1.9673 - learning_rate: 0.0010

Epoch 47/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9062 - loss:
1.3851 - val_accuracy: 0.3750 - val_loss: 1.9469 - learning_rate: 0.0010

Epoch 48/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8750 - loss:
1.4196 - val_accuracy: 0.3750 - val_loss: 1.9497 - learning_rate: 0.0010

Epoch 49/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9375 - loss:
1.3120 - val_accuracy: 0.5000 - val_loss: 1.9516 - learning_rate: 0.0010

Epoch 50/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8438 - loss:
1.3602 - val_accuracy: 0.5000 - val_loss: 1.9420 - learning_rate: 0.0010

Epoch 51/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9062 - loss:
1.2796 - val_accuracy: 0.5000 - val_loss: 1.9189 - learning_rate: 0.0010

Epoch 52/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **20s** 20s/step - accuracy: 0.8125 - loss:
1.3907 - val_accuracy: 0.3750 - val_loss: 1.9078 - learning_rate: 0.0010

Epoch 53/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9375 - loss:
1.2314 - val_accuracy: 0.3750 - val_loss: 1.8876 - learning_rate: 0.0010

Epoch 54/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8750 - loss:
1.2974 - val_accuracy: 0.3750 - val_loss: 1.8533 - learning_rate: 0.0010

Epoch 55/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8750 - loss:
1.2971 - val_accuracy: 0.3750 - val_loss: 1.8168 - learning_rate: 0.0010

Epoch 56/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **13s** 13s/step - accuracy: 0.8125 - loss:
1.2886 - val_accuracy: 0.3750 - val_loss: 1.7623 - learning_rate: 0.0010

Epoch 57/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9375 - loss:
1.1362 - val_accuracy: 0.3750 - val_loss: 1.7361 - learning_rate: 0.0010

Epoch 58/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8125 - loss:
1.2506 - val_accuracy: 0.3750 - val_loss: 1.7067 - learning_rate: 0.0010

Epoch 59/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8438 - loss:
1.1959 - val_accuracy: 0.3750 - val_loss: 1.6791 - learning_rate: 0.0010

Epoch 60/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **20s** 20s/step - accuracy: 0.7500 - loss:
1.3282 - val_accuracy: 0.3750 - val_loss: 1.6786 - learning_rate: 0.0010

Epoch 61/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **13s** 13s/step - accuracy: 0.8125 - loss:
1.2185 - val_accuracy: 0.3750 - val_loss: 1.6778 - learning_rate: 0.0010

Epoch 62/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.8750 - loss:
1.1372 - val_accuracy: 0.3750 - val_loss: 1.6777 - learning_rate: 0.0010

Epoch 63/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9375 - loss:
1.1199 - val_accuracy: 0.3750 - val_loss: 1.6801 - learning_rate: 0.0010

Epoch 64/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9375 - loss:
1.0894 - val_accuracy: 0.3750 - val_loss: 1.6833 - learning_rate: 0.0010

Epoch 65/100

**1/1** ━━━━━━━━━━━━━━━━━━━━ **12s** 12s/step - accuracy: 0.9688 - loss:
1.0512 - val_accuracy: 0.3750 - val_loss: 1.6814 - learning_rate: 0.0010

In \[17\]:

val_loss,val_acc=model.evaluate(validation_generator)

print(f'validation
loss:**{**val_loss**}\n**validation_accuracy:**{**val_acc**}**')

**1/1** ━━━━━━━━━━━━━━━━━━━━ **2s** 2s/step - accuracy: 0.3750 - loss:
1.6777

validation loss:1.6776763200759888

validation_accuracy:0.375

In \[18\]:

history_df = pd.DataFrame(history.history)

history_df.loc\[:, \['loss', 'val_loss'\]\].plot();

print("Minimum validation loss:
**{}**".format(history_df\['val_loss'\].min()))

Minimum validation loss: 1.6776762008666992

Prediction

In \[19\]:

import glob

from sklearn.utils import shuffle

import matplotlib.pyplot as plt

In \[20\]:

IMG_DIM = (224, 224)

*\# Read in all O and R test images file paths. Shuffle and select 50
random test images.*

test_files = glob.glob('/kaggle/working/plates/test/\*.jpg') *\# Adjust
file extension as per your image format*

test_files = shuffle(test_files)\[:50\] *\# Shuffle and select 50 random
images*

*\# Extract images and labels*

test_imgs = \[\]

test_labels = \[\]

for img **in** test_files:

*\# Load and resize image*

img_array =img_to_array(load_img(img, target_size=IMG_DIM))

test_imgs.append(img_array)

*\# Convert lists to numpy arrays*

test_imgs = np.array(test_imgs)

test_labels = np.array(test_labels)

*\# Standardize*

test_imgs_scaled = test_imgs.astype('float32') / 255.0

*\# Define functions for label conversion*

def class2num(label):

return 0 if label == 'cleaned' else 1

def num2class(num):

return 'cleaned' if num \< 0.5 else 'dirty'

*\# Convert labels to numerical format for model training*

test_labels_enc = np.array(\[class2num(label) for label **in**
test_labels\])

*\# Predict using the model*

predictions_model_history = model.predict(test_imgs_scaled, verbose=0)

*\# Convert predictions from probabilities to class labels*

predictions_model_labels = np.array(\[num2class(pred\[0\]) for pred
**in** predictions_model_history\])

*\# Example of predicting on a single custom image*

custom_im = test_imgs_scaled\[2\] *\# Assuming index 2 is a valid index
in your test set*

plt.imshow(custom_im)

plt.axis('off')

plt.show()

*\# Reshape and predict with the model*

custom_im = custom_im.reshape((1, IMG_DIM\[0\], IMG_DIM\[1\], 3)) *\#
Reshape to match model input shape*

prediction_custom_im = model.predict(custom_im, verbose=0)

*\# If predictions are probabilities, use the following line*

print(f"Prediction for custom image:
**{**num2class(prediction_custom_im\[0\]\[0\])**}**")

Prediction for custom image: dirty

In \[21\]:

preds = model.predict(test_generator, verbose=True)

labels = \['dirty' if x \> 0.5 else 'cleaned' for x **in** preds\]

submission_df =
pd.read_csv('/kaggle/input/platesv2/sample_submission.csv')

submission_df\['label'\] = labels

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121:
UserWarning: Your \`PyDataset\` class should call
\`super().\_\_init\_\_(\*\*kwargs)\` in its constructor. \`\*\*kwargs\`
can include \`workers\`, \`use_multiprocessing\`, \`max_queue_size\`. Do
not pass these arguments to \`fit()\`, as they will be ignored.

self.\_warn_if_super_not_called()

**744/744** ━━━━━━━━━━━━━━━━━━━━ **212s** 285ms/step

In \[22\]:

submission_df.to_csv('submission.csv', index=False)